In [3]:
import pandas as pd

In [173]:
df1 = pd.read_excel("../data/raw_data/用电负荷(2019080910).xlsx")

In [174]:
df1.head(2)

,序号,用电企业名称,日期,00:00,01:00,02:00,03:00,04:00,05:00,06:00,...,14:00,15:00,16:00,17:00,18:00,19:00,20:00,21:00,22:00,23:00
0,32,肇庆市现代筑美家居有限公司,2019-08-01,0.06,0.18,0.18,0.18,0.18,0.18,0.12,...,0.9,0.96,0.9,0.36,0.84,0.90,0.78,0.66,0.72,0.78
1,33,肇庆市现代筑美家居有限公司,2019-08-02,0.36,0.60,0.78,0.72,0.72,0.72,0.72,...,0.9,0.96,0.9,0.42,0.78,0.96,0.78,0.78,0.78,0.78


In [175]:
df1.drop(["序号","用电企业名称"],axis=1, inplace=True)

In [176]:
df1.set_index(["日期"], inplace=True)

In [177]:
df2=df1.stack().reset_index()

In [178]:
df2

,日期,level_1,0
0,2019-08-01,00:00,0.060
1,2019-08-01,01:00,0.180
2,2019-08-01,02:00,0.180
3,2019-08-01,03:00,0.180
4,2019-08-01,04:00,0.180
5,2019-08-01,05:00,0.180
6,2019-08-01,06:00,0.120
7,2019-08-01,07:00,0.360
8,2019-08-01,08:00,0.780
9,2019-08-01,09:00,0.900


In [179]:
def half_month(day):
    return 1 if day < 15 else 2
def holiday(month, day):
    if ((month == 9) and (day==13 or day==14 or day==15)) or ((month == 10) and (day==1 or day==2 or day==3 or day==4 or day==5 or day==6 or day==7)):
        return 1 
    else:
        return 2


df2.loc[:,'year'] = df2['日期'].apply(lambda x:x.year)
df2.loc[:,'month'] = df2['日期'].apply(lambda x:x.month)
df2.loc[:,'day'] = df2['日期'].apply(lambda x: x.day)
df2.loc[:,'day_of_week'] = df2['日期'].apply(lambda x: x.dayofweek)
df2.loc[:,'weekend'] = df2['日期'].apply(lambda x:x.dayofweek>4)
df2.loc[:,'half_month'] = df2['日期'].apply(lambda x: half_month(x.day))
df2.loc[:,'holiday'] = df2['日期'].apply(lambda x: holiday(x.month, x.day))

In [180]:
df2["day_of_week"].value_counts()

3    58533
2    54552
1    54528
5    54431
6    54408
0    54408
4    54360
Name: day_of_week, dtype: int64

In [181]:
df2=df2.drop('level_1', axis=1).join(df2['level_1'].str.split(':', expand=True).stack().reset_index(level=1, drop=True).rename('hour'))

In [182]:
#df2["hour"].astype("int64")# 直接转转不了 先转str 再转 int
df2["hour"]=df2["hour"].astype(str).astype(int)
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 770440 entries, 0 to 385219
Data columns (total 10 columns):
日期             770440 non-null datetime64[ns]
0              770440 non-null float64
year           770440 non-null int64
month          770440 non-null int64
day            770440 non-null int64
day_of_week    770440 non-null int64
weekend        770440 non-null bool
half_month     770440 non-null int64
holiday        770440 non-null int64
hour           770440 non-null int64
dtypes: bool(1), datetime64[ns](1), float64(1), int64(7)
memory usage: 59.5 MB


In [183]:
df2

,日期,0,year,month,day,day_of_week,weekend,half_month,holiday,hour
0,2019-08-01,0.060,2019,8,1,3,False,1,2,0
0,2019-08-01,0.060,2019,8,1,3,False,1,2,0
1,2019-08-01,0.180,2019,8,1,3,False,1,2,1
1,2019-08-01,0.180,2019,8,1,3,False,1,2,0
2,2019-08-01,0.180,2019,8,1,3,False,1,2,2
2,2019-08-01,0.180,2019,8,1,3,False,1,2,0
3,2019-08-01,0.180,2019,8,1,3,False,1,2,3
3,2019-08-01,0.180,2019,8,1,3,False,1,2,0
4,2019-08-01,0.180,2019,8,1,3,False,1,2,4
4,2019-08-01,0.180,2019,8,1,3,False,1,2,0


In [184]:
df2.rename(columns={0:'负荷'},inplace=True) 

In [185]:
df2.drop(["日期","year", "month"],axis=1,inplace=True)

In [186]:
df2.drop(["hour"],axis=1,inplace=True)

In [187]:
df3=df2.groupby(["day","day_of_week","weekend","half_month","holiday"]).sum()

In [188]:
df3.reset_index()

,day,day_of_week,weekend,half_month,holiday,负荷
0,1,1,False,1,1,2804.974124
1,1,3,False,1,2,4241.474366
2,1,6,True,1,2,4492.914558
3,2,0,False,1,2,4843.282744
4,2,2,False,1,1,3680.050844
5,2,4,False,1,2,4452.600162
6,3,1,False,1,2,4875.074300
7,3,3,False,1,1,3901.887190
8,3,5,True,1,2,4505.448820
9,4,2,False,1,2,4999.165572


In [189]:
df3.to_csv("./feature_data_without_hour.csv")

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
enc = OneHotEncoder()

In [171]:
from

ImportError: cannot import name 'xgb' from 'xgboost' (/Users/a1/anaconda3/lib/python3.7/site-packages/xgboost/__init__.py)